In [1]:
import pandas as pd 
import numpy as np
import requests
import json

archivo = open("C:/Users/Nur/Documents/tokens/tmdb_token.txt", "r")
contenido = archivo.read()
archivo.close()
token = contenido

In [6]:
df_best_picture = pd.read_csv("./data/best_picture.csv")
df_best_picture

,year_film,year_ceremony,ceremony,name,film,winner
0,1927,1928,1,The Caddo Company,The Racket,False
1,1927,1928,1,Fox,7th Heaven,False
2,1927,1928,1,Paramount Famous Lasky,Wings,True
3,1928,1929,2,Feature Productions,Alibi,False
4,1928,1929,2,Fox,In Old Arizona,False
...,...,...,...,...,...,...
586,2022,2023,95,"Kristie Macosko Krieger, Steven Spielberg and ...",The Fabelmans,False
587,2022,2023,95,"Todd Field, Alexandra Milchan and Scott Lamber...",Tár,False
588,2022,2023,95,"Tom Cruise, Christopher McQuarrie, David Ellis...",Top Gun: Maverick,False
589,2022,2023,95,"Erik Hemmendorff and Philippe Bober, Producers",Triangle of Sadness,False


# INFO

- [ ] cruzar con tmdb, rotten tomatoes, etc
- [ ] oscar bait
- [ ] [IMDB academy awards](https://www.imdb.com/event/ev0000003/2023/1/?ref_=ev_eh)
- [ ] [Cinema timeline](https://www.timetoast.com/timelines/evolution-of-movies-75835972-1771-4afa-8a8c-6b3356decbf6)
- [ ] [AI Movie Companion for Ratings](https://routinehub.co/shortcut/15932/)

# PRUEBAS

## [TMDB API](https://developer.themoviedb.org/reference/search-movie)

Request
- he dividido df_best_picture en lotes porque la api permite unos 50 requests por segundos
- he obtenido los datos de la pelicula por titulo y año
- se han añadido los datos de cada película a una lista de jsons

In [8]:
movie_data = []
base_url = "https://api.themoviedb.org/3/search/movie?"

batches = np.array_split(df_best_picture, 11)  # Dividir en lotes
for batch in batches:
    for index, row in batch.iterrows():
        # Obtener el nombre de la película y el año de la fila actual
        name = row['film'].replace(" ", "%20")
        year = row['year_film']

        # Request
        url = f"{base_url}query={name}&include_adult=false&language=en-US&page=1&year={year}"

        headers = {
            "accept": "application/json",
            "Authorization": f"Bearer {token}"
        }
        response = requests.get(url, headers=headers)

        # Buscar la primera coincidencia exacta en los resultados
        found = False
        for result in response.json()['results']:
            if result['title'].lower() == row['film'].lower():
                movie_data.append(result)
                found = True
                break  # Salir del bucle después de encontrar la primera coincidencia

        # Si no se encuentra coincidencia exacta le sumo 1 al año
        if not found:
            year = row['year_film']+1

            url = f"{base_url}query={name}&include_adult=false&language=en-US&page=1&year={year}"

            headers = {
                "accept": "application/json",
                "Authorization": f"Bearer {token}"
            }
            response = requests.get(url, headers=headers)

            found = False
            for result in response.json()['results']:
                if result['title'].lower() == row['film'].lower():
                    movie_data.append(result)
                    found = True
                    break
            
            # Si sigue sin encontrarse le resto 1 al año
            if not found:
                year = row['year_film']-1

                url = f"{base_url}query={name}&include_adult=false&language=en-US&page=1&year={year}"

                headers = {
                    "accept": "application/json",
                    "Authorization": f"Bearer {token}"
                }
                response = requests.get(url, headers=headers)

                found = False
                for result in response.json()['results']:
                    if result['title'].lower() == row['film'].lower():
                        movie_data.append(result)
                        found = True
                        break
                # Para poder ver que peli exactamente es la que no me encuentra los datos    
                if not found:
                    movie_data.append({"title": f"{row['film']}",
                                    "release_date": f"{row['year_film']}"})


c:\Users\Nur\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
movie_data

[{'adult': False,
  'backdrop_path': '/yCpYVgJH7d49vDc44SKrHLEVhcq.jpg',
  'genre_ids': [80, 18],
  'id': 27777,
  'original_language': 'en',
  'original_title': 'The Racket',
  'overview': 'A renegade police captain sets out to catch a sadistic mob boss.',
  'popularity': 2.635,
  'poster_path': '/wHAJfHslSb6OpNQmj1hu9aE9IQR.jpg',
  'release_date': '1928-11-01',
  'title': 'The Racket',
  'video': False,
  'vote_average': 6.4,
  'vote_count': 29},
 {'adult': False,
  'backdrop_path': '/ebgjvr2dwubTYzOfa419bAzumQ0.jpg',
  'genre_ids': [18, 10749],
  'id': 82474,
  'original_language': 'en',
  'original_title': '7th Heaven',
  'overview': 'A dejected Parisian sewer worker feels his prayers have been answered when he falls in love with a street waif.',
  'popularity': 7.361,
  'poster_path': '/3ETXMu5PrcVMRJpfN2Z3MwcfXzy.jpg',
  'release_date': '1927-09-10',
  'title': '7th Heaven',
  'video': False,
  'vote_average': 7.317,
  'vote_count': 90},
 {'adult': False,
  'backdrop_path': '/soJ

In [10]:
df_movie_data = pd.DataFrame(movie_data)
df_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              591 non-null    bool   
 1   backdrop_path      591 non-null    object 
 2   genre_ids          591 non-null    object 
 3   id                 591 non-null    int64  
 4   original_language  591 non-null    object 
 5   original_title     591 non-null    object 
 6   overview           591 non-null    object 
 7   popularity         591 non-null    float64
 8   poster_path        591 non-null    object 
 9   release_date       591 non-null    object 
 10  title              591 non-null    object 
 11  video              591 non-null    bool   
 12  vote_average       591 non-null    float64
 13  vote_count         591 non-null    int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 56.7+ KB


In [11]:
df_movie_data

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/yCpYVgJH7d49vDc44SKrHLEVhcq.jpg,"[80, 18]",27777,en,The Racket,A renegade police captain sets out to catch a ...,2.635,/wHAJfHslSb6OpNQmj1hu9aE9IQR.jpg,1928-11-01,The Racket,False,6.400,29
1,False,/ebgjvr2dwubTYzOfa419bAzumQ0.jpg,"[18, 10749]",82474,en,7th Heaven,A dejected Parisian sewer worker feels his pra...,7.361,/3ETXMu5PrcVMRJpfN2Z3MwcfXzy.jpg,1927-09-10,7th Heaven,False,7.317,90
2,False,/soJGAjMOnvHaWZvAZm7r5OMTpM4.jpg,"[18, 28, 10752, 10749]",28966,en,Wings,"Two young men, one rich, one middle class, bot...",11.324,/kEl6KCBgdmT1Nex3ka0EIWAOmtm.jpg,1927-08-12,Wings,False,7.198,248
3,False,/oWD7JsI3mdJ8A4fZTs5GymqoeOr.jpg,"[28, 80, 53]",13847,en,Alibi,"Chick Williams, a prohibition gangster, rejoin...",1.960,/2C3ZDxVrOVafzNoOVrxsNQDiHey.jpg,1929-04-20,Alibi,False,5.656,16
4,False,/VRjzMGnGuAXycd1alavkwgAqQJ.jpg,"[28, 37]",83989,en,In Old Arizona,Army Sergeant Mickey Dunn sets out in pursuit ...,4.693,/x1XSQeLUdoi72P5UOXqAyL46B9a.jpg,1928-12-25,In Old Arizona,False,5.208,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,False,/2OnmtVOEpCwt0sYtss5kNxyYM2W.jpg,"[18, 35]",804095,en,The Fabelmans,"Growing up in post-World War II era Arizona, y...",36.046,/h7llKkqkkJtJrTOaDLuVeUYDQ7I.jpg,2022-11-11,The Fabelmans,False,7.692,1946
587,False,/84XcRwKHAw4VXdKOYTSW5ARxFEt.jpg,"[18, 10402]",817758,en,TÁR,Renowned musician Lydia Tár is days away from ...,36.108,/dRVAlaU0vbG6hMf2K45NSiIyoUe.jpg,2022-09-23,TÁR,False,7.143,1007
588,False,/AaV1YIdWKnjAIAOe8UUKBFm327v.jpg,"[28, 18]",361743,en,Top Gun: Maverick,After more than thirty years of service as one...,193.433,/62HCnUTziyWcpDaBO2i1DX17ljH.jpg,2022-05-24,Top Gun: Maverick,False,8.237,8042
589,False,/jENycYiYGr5NvMs789zj06JkW5l.jpg,"[35, 18]",497828,en,Triangle of Sadness,A celebrity model couple are invited on a luxu...,44.240,/k9eLozCgCed5FGTSdHu0bBElAV8.jpg,2022-09-18,Triangle of Sadness,False,7.116,1782


In [13]:
df_movie_data.drop(columns=["adult", "backdrop_path", "id", "poster_path", "video", "vote_count"], inplace=True)

In [14]:
df_movie_data.drop(columns="popularity", inplace=True)

In [15]:
df_movie_data

,genre_ids,original_language,original_title,overview,release_date,title,vote_average
0,"[80, 18]",en,The Racket,A renegade police captain sets out to catch a ...,1928-11-01,The Racket,6.400
1,"[18, 10749]",en,7th Heaven,A dejected Parisian sewer worker feels his pra...,1927-09-10,7th Heaven,7.317
2,"[18, 28, 10752, 10749]",en,Wings,"Two young men, one rich, one middle class, bot...",1927-08-12,Wings,7.198
3,"[28, 80, 53]",en,Alibi,"Chick Williams, a prohibition gangster, rejoin...",1929-04-20,Alibi,5.656
4,"[28, 37]",en,In Old Arizona,Army Sergeant Mickey Dunn sets out in pursuit ...,1928-12-25,In Old Arizona,5.208
...,...,...,...,...,...,...,...
586,"[18, 35]",en,The Fabelmans,"Growing up in post-World War II era Arizona, y...",2022-11-11,The Fabelmans,7.692
587,"[18, 10402]",en,TÁR,Renowned musician Lydia Tár is days away from ...,2022-09-23,TÁR,7.143
588,"[28, 18]",en,Top Gun: Maverick,After more than thirty years of service as one...,2022-05-24,Top Gun: Maverick,8.237
589,"[35, 18]",en,Triangle of Sadness,A celebrity model couple are invited on a luxu...,2022-09-18,Triangle of Sadness,7.116


In [16]:
df_movie_data['release_date'] = pd.to_datetime(df_movie_data['release_date'])

In [17]:
df_movie_data

,genre_ids,original_language,original_title,overview,release_date,title,vote_average
0,"[80, 18]",en,The Racket,A renegade police captain sets out to catch a ...,1928-11-01,The Racket,6.400
1,"[18, 10749]",en,7th Heaven,A dejected Parisian sewer worker feels his pra...,1927-09-10,7th Heaven,7.317
2,"[18, 28, 10752, 10749]",en,Wings,"Two young men, one rich, one middle class, bot...",1927-08-12,Wings,7.198
3,"[28, 80, 53]",en,Alibi,"Chick Williams, a prohibition gangster, rejoin...",1929-04-20,Alibi,5.656
4,"[28, 37]",en,In Old Arizona,Army Sergeant Mickey Dunn sets out in pursuit ...,1928-12-25,In Old Arizona,5.208
...,...,...,...,...,...,...,...
586,"[18, 35]",en,The Fabelmans,"Growing up in post-World War II era Arizona, y...",2022-11-11,The Fabelmans,7.692
587,"[18, 10402]",en,TÁR,Renowned musician Lydia Tár is days away from ...,2022-09-23,TÁR,7.143
588,"[28, 18]",en,Top Gun: Maverick,After more than thirty years of service as one...,2022-05-24,Top Gun: Maverick,8.237
589,"[35, 18]",en,Triangle of Sadness,A celebrity model couple are invited on a luxu...,2022-09-18,Triangle of Sadness,7.116


In [18]:
df_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   genre_ids          591 non-null    object        
 1   original_language  591 non-null    object        
 2   original_title     591 non-null    object        
 3   overview           591 non-null    object        
 4   release_date       591 non-null    datetime64[ns]
 5   title              591 non-null    object        
 6   vote_average       591 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 32.4+ KB


In [23]:
df_movie_data.to_csv("./data/movie_data.csv", index=False)

### Genero

Obtengo la lista de géneros de TMDB

In [19]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}"
}

response = requests.get(url, headers=headers)

genres = response.json()

In [28]:
with open("./data/genres.json", 'w') as g:
    json.dump(genres, g)